# Data Preprocessing&Impute

### 数据无量纲化:不同规格数据转到统一规格，不同分布转为某个特定分布
比如：1. 以梯度和矩阵为核心的算法：逻辑回归，SVM，神经网络，无量纲化可加快求解速度，
      2. 距离类模型: KNN，K-means聚类。无量钢化提升精度。避免某一取值范围特别大的特征对距离计算造成影响。
      3. 特例: 决策树和树的集成算法中，对决策树不做无量纲化，Decision tree 可以将任意数据处理得很好。
      --------------                                                   -------------
### 分类
- 线性
    + 中心化 Zero-centered 或者Mean-subtraction (本质是所有记录减去一个固定值，即让数据样本数据平移到某个位置。)
    + 缩放处理 Scale(本质是通过除以一个固定值，将数据固定在某个范围中，取对数也是一种缩放处理)
- 非线性 
min


### preprocessing.MinMaxScaler
数据(x)按照最小值中心化后，再按极差(MAX-MIN)缩放。数据移动了最小值个单位，并会被收敛到\[0,1\]之间，这个过程交**数据归一化(Normalization又叫Min-Max-Scaling)**。区分:正则化(regularization)不是预处理得手段，归一化后得数据服从正态分布。



+ preprocessing.MinMaxscaler实现Normalization。
+ 参数feature_range：控制数据压缩到得范围，默认\[0,1\]

In [2]:
from sklearn.preprocessing import MinMaxScaler#导库
data = [[-1,2],[-0.5,6],[0,10],[1,18]]
#换成表
import pandas as pd
pd.DataFrame(data)

,0,1
0,-1.0,2
1,-0.5,6
2,0.0,10
3,1.0,18


In [3]:
#Normalization
scaler = MinMaxScaler()#实例化
scaler = scaler.fit(data)#fit,此处是生成min(x)与max(x)
result = scaler.transform(data)#通过接口导出结果
result

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [4]:
result_ = scaler.fit_transform(data)#训练与导出结果一步达成
result_

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [5]:
scaler.inverse_transform(result)#将归一化得结果逆转#还原

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

In [6]:
#使用MinMaxScaler的参数feature_range将数据归一化到[0,1]以外的范围
data = [[-1,2],[-0.5,6],[0,10],[1,18]]
scaler = MinMaxScaler(feature_range=[5,10])
result = scaler.fit_transform(data)#一步导出结果
result

array([[ 5.  ,  5.  ],
       [ 6.25,  6.25],
       [ 7.5 ,  7.5 ],
       [10.  , 10.  ]])

> 当X中的特征数量非常多时，fit报错显示数据量大了俺计算不了，此时应用**partial_fit**作为训练接口。

### scaler = scaler.partial_fit(data)

## Normalization实现

In [7]:
import numpy as np
X = np.array([[-1,2],[-0.5,6],[0,10],[1,18]])
X.min(axis=0)#跨列计算
# 归一化
X_nor = (X-X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))
X_nor

array([[0.  , 0.  ],
       [0.25, 0.25],
       [0.5 , 0.5 ],
       [1.  , 1.  ]])

In [8]:
#逆转归一化
X_returned = X_nor * (X.max(axis=0)-X.min(axis=0))+X.min(axis=0)
X_returned

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

# preprocessing.StandardScaler
数据(x)按均值(μ)中心化后，按标准差(σ)缩放，数据会服从均值为 0 方差为 1 的标准正态分布，此过程，叫**数据标准化(Standardization又叫Z-score normalization)**




In [16]:
data = [[-1,2],[-0.5,6],[0,10],[1,18]]
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()#实例化
scaler.fit(data)#fit，本质是生成均值和/方差

StandardScaler(copy=True, with_mean=True, with_std=True)

In [18]:
scaler.mean_ #查看均值的属性mean_# 自动是按一整列

array([-0.125,  9.   ])

In [19]:
scaler.var_#查看方差

array([ 0.546875, 35.      ])

In [20]:
x_std = scaler.transform(data)#通过接口导出结果
x_std

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [21]:
x_std.mean()#导出结果是一个数组，用mean()查看均值

0.0

In [22]:
x_std.std()#用std()查看方差

1.0

In [23]:
scaler.fit_transform(data)#使用fit_transform(data)一步达成结果

array([[-1.18321596, -1.18321596],
       [-0.50709255, -0.50709255],
       [ 0.16903085,  0.16903085],
       [ 1.52127766,  1.52127766]])

In [24]:
scaler.inverse_transform(x_std)#使用inverse_transform逆转标准化

array([[-1. ,  2. ],
       [-0.5,  6. ],
       [ 0. , 10. ],
       [ 1. , 18. ]])

+ 对于StandardScaler(标准化)和MinMaxScaler(归一化)，NaN被视为缺失值，fit时直接忽略，在transform时会继续保持NaN的形式。
+ 尽管去量纲化不是具体算法，但在fit接口中，依然只允许导入至少二维数组，一维数组导入报错
(通常X是特征矩阵，现实中feature matrix不太可能一维)
+ 练习时用大于1-dim的，1-dim的要reshape(-1,1)改维度。

+ 因为MinMaxScaler对异常值非常敏感，在PCA，聚类，逻辑回归，支持向量机，神经网络中StandardScaler往往最好
+ MinMaxScaler在不涉及距离度量，梯度，协方差计算以及数据需要被压缩到特定区间时使用广泛，比如数字图像处理中量化像素强度时，都会使用MinMaxScaler将数据压缩于[0,1]区间之中。
+ 先试试StandardScaler,效果不好用MinMaxScaler

##### sklearn也提供除StandardScaler和MinMaxScaler之外的各种**缩放处理**(中心化只需要pandas广播减去某个数就好，因此sklearn不提供任何中心化功能)。
+ 在希望压缩数据却不影响数据的稀疏性(矩阵中取值为0的个数)时，使用MaxAbsScaler只压缩，不中心化
+ 异常值多噪声很大时，可能选分位数来无量纲化，此时用RobustScaler